In [1]:
library(data.table)
library(dplyr)


Attaching package: ‘dplyr’

The following objects are masked from ‘package:data.table’:

    between, first, last

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



In [2]:
rda <- '/oak/stanford/groups/mrivas/projects/PRS/private_output/snpnet_biomarker_PRS/biomarkers_20190406/3_snpnet/Alkaline_phosphatase/results/output_iter_1.rda'


In [3]:
fit <- readRDS(rda)

In [5]:
source('/oak/stanford/groups/mrivas/users/ytanigaw/repos/rivas-lab/PRS/helper/snpnet-fit.R')

Loading snpnet
Loading required package: Rcpp


ERROR: Error in .stop(output, "parse error:"): parse error:
usage: PROGRAM [-h] -p P -n N -g G -o O [-b b] [-f f] [--covarsList C]
               [--cpu t] [--mem m] [--nPCs c] [--prevIter i] [--bim B]
               [--rds r]
PROGRAM: error: the following arguments are required: -p, -n, -g, -o


In [6]:
df <- snpnet_fit2table(fit)

Warning message:
“Expected 2 pieces. Missing pieces filled with `NA` in 48 rows [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, ...].”

In [8]:
df %>% dim()

[1] 1127    3

In [9]:
df %>% tail()

,ID,A1,BETA
1122,rs616718,A,0
1123,rs4581844,C,0
1124,rs12624577,C,0
1125,rs2414078,G,0
1126,rs5832844,T,0
1127,rs2202955,G,0


In [10]:
df %>% filter(BETA != 0) %>% dim()

[1] 123   3

In [11]:
# beta for covariates
df_covars <- snpnet_extract_covars(df, covariates)    
# df_covars %>% write.table(
#     paste0(args$b, '.covars.tsv'), 
#     quote=FALSE, row.names=FALSE, sep='\t'
# )    


ERROR: Error in data.frame(ID = covariates): object 'covariates' not found


In [13]:
covariates <- read_file_as_list('/oak/stanford/groups/mrivas/users/ytanigaw/repos/rivas-lab/PRS/notebook/20190406_biomarker_covar/biomarker_covar.colnames')



In [15]:
# beta for covariates
df_covars <- snpnet_extract_covars(df, covariates)    
# df_covars %>% write.table(
#     paste0(args$b, '.covars.tsv'), 
#     quote=FALSE, row.names=FALSE, sep='\t'
# )    


Joining, by = "ID"
Warning message:
“Column `ID` joining character vector and factor, coercing into character vector”

In [21]:
snpnet_fit2table <- function(fit) {
    df <- fit$beta[which.max(fit$metric.val)] %>% data.frame()
    colnames(df) <- 'BETA'

    df_separated <- df %>%
    rownames_to_column("feature") %>%
    separate(feature, into=c('ID', 'A1'), sep='_')

    return(df_separated)
}

In [27]:
df <- fit$beta[which.max(fit$metric.val)] %>% data.frame()
colnames(df) <- 'BETA'
df <- df %>% rownames_to_column("feature")

In [35]:
snpnet_fit2table <- function(fit) {
    df <- fit$beta[which.max(fit$metric.val)] %>% data.frame()
    colnames(df) <- 'BETA'
    df %>%
    rownames_to_column("ID") 
}

In [36]:
df <- snpnet_fit2table(fit)

In [37]:
df %>% filter(
    ID %in% covariates
) %>% dim()


[1] 127   2

In [39]:
df %>% filter(
    ! ID %in% covariates
) %>% filter(
    BETA != 0
) %>% separate(
    ID, into=c('ID', 'A1'), sep='_'
)


ID,A1,BETA
rs1135173,A,0.001283684


In [42]:
bim.file <- '/oak/stanford/groups/mrivas/ukbb24983/cal/pgen/ukb24983_cal_cALL_v2.bim'


In [ ]:
    # beta for genotypes
    df_genotypes <- snpnet_join_with_bim(df, args$bim)
    df_genotypes %>% write.table(
        paste0(args$b, '.tsv'), 
        quote=FALSE, row.names=FALSE, sep='\t'
    )
    # beta for covariates
    df_covars <- snpnet_extract_covars(df, covariates)    
    df_covars %>% write.table(
        paste0(args$b, '.covars.tsv'), 
        quote=FALSE, row.names=FALSE, sep='\t'
    )    

In [43]:
# Define constants
bim.cols <- c('CHROM', 'ID', 'CM', 'POS', 'ALT', 'REF')
out.cols <- c('CHROM', 'POS', 'ID', 'REF', 'ALT', 'A1', 'BETA')
# read PLINK bim file
bim.df <- fread(bim.file, data.table = FALSE)
colnames(bim.df) <- bim.cols


In [47]:
snpnet_join_with_bim <- function(df, covariates, bim.file) {
# Define constants
    bim.cols <- c('CHROM', 'ID', 'CM', 'POS', 'ALT', 'REF')
    out.cols <- c('CHROM', 'POS', 'ID', 'REF', 'ALT', 'A1', 'BETA')
# read PLINK bim file
    bim.df <- fread(bim.file, data.table = FALSE)
    colnames(bim.df) <- bim.cols
# process the given data frame (join w/ bim)
    df_joined <- df %>%
    filter(
        ! ID %in% covariates
    ) %>% filter(
        BETA != 0
    ) %>% separate(
        ID, into=c('ID', 'A1'), sep='_'
    ) %>%
    drop_na() %>%
    left_join(bim.df, on='ID') %>%
    select(out.cols) %>%
    arrange(CHROM, POS)
# return
    return(df_joined)
}

In [48]:
df %>% snpnet_join_with_bim(covariates, bim.file)

Joining, by = "ID"


CHROM,POS,ID,REF,ALT,A1,BETA
2,234054873,rs1135173,G,A,A,0.001283684
